# Global Stuffs

In [4]:
# use env bl17_2 to run this script

import param
import panel as pn

import holoviews as hv
import imageio
import os, glob, time

import json
import pandas as pd

import tqdm
import numpy as np


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

In [5]:
def set_plot_style(axs, fonts, xlabel, ylabel):
    axs.set_xlabel(xlabel, fontsize=fonts)
    axs.set_ylabel(ylabel, fontsize=fonts)
    axs.tick_params(axis='both', which='major', direction='out', length=4, width=1)
    axs.tick_params(which='minor', width=1, size=2)  # Adjust size as needed
    axs.minorticks_on() # Turn on minor ticks
    
    #axs.grid(False, which='both', axis='both', linestyle='--', linewidth=0.5)
    axs.set_facecolor('white')
    axs.spines['top'].set_linewidth(1)
    axs.spines['right'].set_linewidth(1)
    axs.spines['bottom'].set_linewidth(1)
    axs.spines['left'].set_linewidth(1)
    axs.tick_params(axis='x', labelsize=fonts)
    axs.tick_params(axis='y', labelsize=fonts)

    return axs

In [3]:
from tkinter import font
import numpy as np
import matplotlib.pyplot as plt
import pyFAI

import numpy as np
import pyFAI
import os
import matplotlib.pyplot as plt
import re

def plot_2d_images(image, poni_file, output_folder, tif_file, azm_range, vmin=None, vmax=None):

    # Load poni file to extract calibration parameters
    ai = pyFAI.load(poni_file)
    pixel_size = ai.pixel1  # Assuming square pixels
    detector_distance = ai.dist

    #print(ai)

    #beamx = 1028.789
    #beamy = 1126.865

    beamx = ai.getFit2D()['centerX']
    beamy = ai.getFit2D()['centerY']


    #print(f"beamx: {beamx}, beamy: {beamy}")

    wavelength = ai.wavelength

    # Convert SAXS to q-space coordinates
    x_pixels = image.shape[1]
    y_pixels = image.shape[0]
    x_coords = np.arange(x_pixels) - beamx
    y_coords = np.arange(y_pixels) - beamy
    xx, yy = np.meshgrid(x_coords, y_coords)

    qx = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    # Calculate azimuthal angle (in degrees) for each pixel relative to the beam center
    azimuthal_angles = np.degrees(np.arctan2(qy, qx))
    
    # Create a mask for the azimuthal angle range
    mask = (azimuthal_angles >= azm_range[0]) & (azimuthal_angles <= azm_range[1])

    # Plot the 2D image with the sector highlighted
    fig, ax = plt.subplots(figsize=(6, 6))
    # set nans to 0
    image = np.nan_to_num(image, nan=0.0)
    # Normalize the image  with sum of intensity
    image = 10000000* (image / np.sum(image))

    


    
    im_avg = ax.pcolormesh(qx, qy, (image), cmap='jet', vmin=vmin, vmax=vmax, shading='auto')
    
    ax.set_aspect('equal')
    
    # Overlay the sector by masking the area outside azimuthal range
    #ax.contourf(qx, qy, mask, levels=[0.5, 1], colors='yellow', alpha=0.5)  # hatches=['//']

    cbar_avg = fig.colorbar(im_avg, ax=ax, shrink=.8, label='Intensity [a.u.]')
    cbar_avg.ax.tick_params(labelsize=20)
    cbar_avg.ax.yaxis.label.set_size(20)
    
    ax.set_xlabel(r"$q_x$ [nm$^{-1}$]", fontsize=20)
    ax.set_ylabel(r"$q_y$ [nm$^{-1}$]", fontsize=20)
    ax.set_title(f"{f'{os.path.splitext(tif_file)[0]}_2D'}", fontsize=14, y=1.05)
    
    ax.tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Save and show the figure
    # creat a Figures folder with output_folder
    output_folder = os.path.join(output_folder, "Figures")
    os.makedirs(output_folder, exist_ok=True)
    # create the folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # save the figure
    plt.savefig(f"{output_folder}/{f'{os.path.splitext(tif_file)[0]}'}.png", dpi=300, bbox_inches='tight')
    plt.close()
    #plt.show()


def sort_files(filenames):
    def extract_numbers(filename):
        match = re.search(r'_s(\d+)_0*(\d+)', filename)
        if match:
            snumber = int(match.group(1))
            number = int(match.group(2))
            return snumber, number
        return float('inf'), float('inf')  # Return large values if the pattern doesn't match

    return sorted(filenames, key=extract_numbers)

def extract_metadata(metadata):
    # Extract the SPEC data
    S = metadata['SPEC']['S']
    i0 = S['1'] if '1' in S else None
    i1 = S['2'] if '2' in S else None
    Photod = S['4'] if '4' in S else None

    # Extract the series_date
    real_date_time = metadata['Eiger_metadata']['series_date']

    return real_date_time, i0, i1, Photod

#i0_1, i1_1, Photod_1 = extract_values(metadata_1)
#i0_2, i1_2, Photod_2 = extract_values(metadata_2)

def extract_metadata(metadata):
    # Extract the SPEC data
    S = metadata['SPEC']['S']
    i0 = S['1'] if '1' in S else None
    i1 = S['2'] if '2' in S else None
    Photod = S['4'] if '4' in S else None

    # Extract the series_date
    real_date_time = metadata['Eiger_metadata']['series_date']

    return real_date_time, i0, i1, Photod

def extract_metadata(metadata):
    timestamp = metadata['timestamp']
    i0 = metadata['i0']
    i1 = metadata['i1']
    Photod = metadata['Photod']

    return timestamp, i0, i1, Photod
#i0_1, i1_1, Photod_1 = extract_values(metadata_1)
#i0_2, i1_2, Photod_2 = extract_values(metadata_2)

In [ ]:
from collections import OrderedDict
from dateutil import parser

import fabio
from silx.io.specfile import SpecFile

import pyFAI
from pyFAI.detectors import Detector
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator

import tifffile
import json

import pandas as pd
import h5py

import warnings
warnings.filterwarnings('ignore')


tiff_fname = r'/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/thresh1_Erick_test_cscan1_Scan00003_s00000_00000.tif'

img_meta = {}
with tifffile.TiffFile(tiff_fname) as tif:
    for tag in tif.pages[0].tags.values():
        name, value = tag.name, tag.value
        if name in img_meta.keys():
            name += '_'
        try:
            img_meta[name] = json.loads(value)
        except:
            pass    

default_motor_names = img_meta['ImageDescription']['SPEC']['Motor_names']
default_counter_names = img_meta['ImageDescription']['SPEC']['Counter_names']


def get_tiff_img_data(fname):
    with tifffile.TiffFile(fname) as tif:
        for idx, page in enumerate(tif.pages):
            if idx == 0:
                img_data = page.asarray().astype(float)
            else:
                img_data = np.concatenate((img_data, page.asarray().astype(float)))

    img_data[img_data > 1e8] = np.nan
    return img_data
                           

def get_tiff_meta_data(fname, df=None):                           
    meta_dict = dict()
    with tifffile.TiffFile(fname) as tif:
        tif_tags = {}
        for tag in tif.pages[0].tags.values():
            name, value = tag.name, tag.value
            if name in meta_dict.keys():
                name += '_'
            try:
                meta_dict[name] = json.loads(value)
            except:
                pass     

    # Get motor scanned
    scan_motor = meta_dict['ImageDescription']['SPEC']['SCAN_COLS']['0']

    # Get time when image acquired
    series_start_time = meta_dict['ImageDescription']['Eiger_metadata']['series_date']
    series_start_time = parser.parse(series_start_time).timestamp()
    img_start_time = series_start_time + meta_dict['ImageDescription']['Eiger_metadata']['start_time']
    
    full_dict = {'timestamp' : img_start_time, 'scan_motor': scan_motor}
        
    try:
        motor_names = meta_dict['ImageDescription']['SPEC']['Motor_names']
    except:
        motor_names = default_motor_names
   
    motor_vals = meta_dict['ImageDescription']['SPEC']['A']

    try:
        counter_names = meta_dict['ImageDescription']['SPEC']['Counter_names']
    except:
        counter_names = default_counter_names
    counter_vals = meta_dict['ImageDescription']['SPEC']['S']
    
    motors = {mname:mval for ((km, mname), (kv, mval)) in zip(motor_names.items(), motor_vals.items())}
    counters = {cname:cval for ((kc, cname), (kc, cval)) in zip(counter_names.items(), counter_vals.items())}
    
    full_dict.update({**motors, **counters})

    try:
        df = pd.concat([df, pd.DataFrame([full_dict])], ignore_index=True)
    except:
        df = pd.DataFrame([full_dict])
    
    return df


# azm and rad integration function: imageio

In [ ]:
import os
import glob
from isort import file
import pyFAI
import numpy as np  # Ensure this is included
import matplotlib.pyplot as plt
import imageio
import fabio
import h5py
import json
from pytools import F
from tqdm import tqdm
import pandas as pd
import re


def integrate_tif_files(base_path, poni_file, mask_file, samp_folder, keyword, output_base_path, nsave2d=3, azm_range = (-60, 60), q1=None, q2=None, vmin=None, vmax=None,plot=False):
    # Locate all 'Threshold' folders
    #print(os.path.join(base_path, samp_folder, 'Threshold*'))
    threshold_folders = glob.glob(os.path.join(base_path, samp_folder, keyword, 'Threshold 1'))
    print(threshold_folders)
    

    if not threshold_folders:
        raise FileNotFoundError("No Threshold folders found.")
    
    # Initialize the PyFAI azimuthal integrator
    ai = pyFAI.load(poni_file)

    # Read the mask file with fabio
    mask = fabio.open(mask_file).data

    # Loop through each threshold folder
    for threshold_folder in threshold_folders:
        intermediate_folder = os.path.basename(os.path.dirname(threshold_folder))
        output_folder = os.path.join(output_base_path, 'OneD_integrated_WAXS_01', samp_folder)#, intermediate_folder)
        save_figures_folder = os.path.join(output_folder, 'Figures')
        os.makedirs(save_figures_folder, exist_ok=True)
        #output_folder = os.path.join(output_base_path, 'oneD_reducted', samp_folder, intermediate_folder)

        #save_2d_folder = os.path.join(output_base_path, 'oneD_reducted','twoD')
        #os.makedirs(save_2d_folder, exist_ok=True)

        #save_1D_folder = os.path.join(output_base_path, 'oneD_reducted','oneD')

        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)

        #txt_folder = os.path.join(output_base_path, 'D_txt')
        #os.makedirs(txt_folder, exist_ok=True)

        # List all .tif files in the current threshold folder
        #tif_files = [f for f in os.listdir(threshold_folder) if f.endswith('.tif')]
        tif_files = [f for f in os.listdir(threshold_folder) if f.endswith('.tif') and not f.startswith('._')]

        
        tif_files = sort_files(tif_files)

        # sort files

        #print(f"tif_files: {tif_files}")

        # Initialize data containers for plotting
        azimuthal_data = []
        radial_data = []
        
        # Use tqdm for a progress bar with a shorter description
        for i, tif_file in enumerate(tqdm(tif_files, desc='Processing', unit='file'), start=1):
            
            file_path = (os.path.join(threshold_folder, tif_file))
            try:
                img = imageio.v3.imread(file_path)
                img[mask == 1] = 0
                #print(img.shape, img.dtype)
                # print (f"img: {img}")
                
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                continue  # Skip the current file and move on to the next


            #an_img = imageio.v3.immeta(file_path)
            #print(file_path)
            #eiger_metadata = json.loads(an_img['description'])
            #eiger_metadata = json.loads(an_img['Eiger_metadata'])
            #print(f"eiger_metadata: {eiger_metadata}")
            
            eiger_metadata_df = get_tiff_meta_data(file_path)

            #print(f"eiger_metadata: {eiger_metadata}")
            
            real_date_time, i0, i1, Photod = extract_metadata(eiger_metadata_df)
            normfactor = Photod * i1

            #print(F"real_date_time: {real_date_time}, i0: {i0}, i1: {i1}, Photod: {Photod}, normfactor: {normfactor}")
            #print(f'real_date_time: {real_date_time}, i0: {i0}, i1: {i1}, Photod: {Photod}, normfactor: {normfactor}')
            #print(f"i1: {i1}, Photod: {Photod}, normfactor: {normfactor}")

            #print(f"real_date_time: {real_date_time}, i0: {i0}, i1: {i1}, Photod: {Photod}, normfactor: {normfactor}")
            
            azmr = azm_range[1] - azm_range[0]
            #print(f"azm_range: {azm_range}, azmr: {azmr}")
            if i % nsave2d == 0:
                plot_2d_images(img, poni_file, output_folder, tif_file, vmin=vmin, vmax=vmax, azm_range=azm_range)

            # Azimuthal integration
            #file_name = os.path.splitext(tif_file)[0] + '_azimuthal.txt'
            result = ai.integrate1d(img, 4000, error_model='poisson', correctSolidAngle=True, azimuth_range = azm_range, normalization_factor=normfactor, polarization_factor=0.95, method='splitpixel', mask=mask)#,filename=os.path.join(txt_folder, file_name))
            q, I_azimuthal, error_azm = result.radial, result.intensity, result.sigma
            azimuthal_data.append((q, I_azimuthal, error_azm, tif_file, real_date_time, i0, i1, Photod))
            
            #plt.imshow(img, cmap='jet')
            #print(f"q: {q}, I_azimuthal: {I_azimuthal}, error_azm: {error_azm}")
            #file_name = os.path.splitext(tif_file)[0] + '_radial'
            # Radial integration
            chi, I_radial = ai.integrate_radial(
                img,
                npt=720, # number of points in the output pattern
                #npt_rad=100,  # number of points in the radial space. Too few points may lead to huge rounding errors.
                radial_range=(q1, q2), 
                azimuth_range=(-180, 0),
                mask=mask,
                normalization_factor=normfactor,
                correctSolidAngle=True,
                polarization_factor=0.95,
                method='splitpixel',
                unit='chi_deg',
                radial_unit='q_nm^-1'
                #filename=os.path.join(output_folder, file_name)
            )
            radial_data.append((chi, I_radial, tif_file, real_date_time, i0, i1, Photod))

            # Combine chi and I_radial into a single array
            #data_to_save = np.column_stack((chi, I_radial))

            # Save the combined data to a single file
            #np.savetxt(os.path.join(txt_folder, file_name + '.txt'), data_to_save, header="Chi, I_radial")

            #print(f"chi: {chi}, I_radial: {I_radial}")
        # Save the azimuthal data in HDF5 format
        
        #output_folder_basename = os.path.basename(output_folder)
        with h5py.File(os.path.join(output_folder, f'{intermediate_folder}_{azmr}_azimuthal_data.h5'), 'w') as hf:
            for idx, (q, I, err, tif_file, real_date_time, i0, i1, Photod) in enumerate(azimuthal_data):
                group = hf.create_group(f'data_{idx}')
                group.create_dataset('q', data=q)
                group.create_dataset('I', data=I)
                group.create_dataset('error', data=err)
                group.create_dataset('filename', data=(tif_file))
                group.create_dataset('real_date_time', data=(real_date_time))
                group.create_dataset('i0', data=np.float32(i0) if i0 is not None else np.nan)
                group.create_dataset('i1', data=np.float32(i1) if i1 is not None else np.nan)
                group.create_dataset('Photod', data=np.float32(Photod) if Photod is not None else np.nan)
        
        # Save the radial data in HDF5 format
        #output_folder_basename = os.path.basename(output_folder)
        with h5py.File(os.path.join(output_folder, f'{intermediate_folder}_radial_data.h5'), 'w') as hf:
            for idx, (chi, I, tif_file, real_date_time, i0, i1, Photod) in enumerate(radial_data):
                group = hf.create_group(f'data_{idx}')
                group.create_dataset('chi', data=chi)
                group.create_dataset('I', data=I)
                group.create_dataset('filename', data=np.bytes_(tif_file))
                group.create_dataset('real_date_time', data=np.bytes_(real_date_time))
                group.create_dataset('i0', data=np.float32(i0) if i0 is not None else np.nan)
                group.create_dataset('i1', data=np.float32(i1) if i1 is not None else np.nan)
                group.create_dataset('Photod', data=np.float32(Photod) if Photod is not None else np.nan)
        
        # Plot the integrated data
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 8))

        #print(f"azimuthal_data: {azimuthal_data}")
        
        num_lines = len(azimuthal_data)
        cmap = plt.cm.viridis  # You can change this to any colormap you like
        norm = plt.Normalize(vmin=0, vmax=num_lines - 1)
        ii =0 
        # Plot azimuthal data
        # condition to plot
        for q, I, err, tif_file, *_ in azimuthal_data:
            color = cmap(norm(ii))
            ax1.plot(q, I, label=tif_file, color=color)
            if q1 is not None:
                ax1.axvline(x=q1, color='r', linestyle='--')
            if q2 is not None:
                ax1.axvline(x=q2, color='r', linestyle='--')
            ii=ii+1
        
        ax1.set_xlabel('q [1/nm]')
        ax1.set_xlim(4, 30)
        ax1.set_ylabel('Intensity [a.u.]')
        ax1.set_title('Azimuthal Integration')

        set_plot_style(ax1, 20, 'q [1/nm]', 'Intensity [a.u.]')
        #ax1.legend()
        ii=0
        # Plot radial data
        for chi, I, tif_file, *_ in radial_data:
            color = cmap(norm(ii))
            ax2.plot(chi, I, 'o', markersize=2, label=tif_file,color=color)
            ii=ii+1
        ax2.set_xlabel('chi [degrees]')
        ax2.set_ylabel('Intensity [a.u.]')
        ax2.set_xlim(-180, 0)
        #ax2.set_title('Radial Integration')
        set_plot_style(ax2, 20, 'chi [degrees]', 'Intensity [a.u.]')
        #ax2.legend()

        plt.tight_layout()
        title_name = [f.replace('thresh1_', '').replace('_s00000_00000.tif', '') for f in tif_files]

        #plt.savefig(os.path.join(save_figures_folder, f'{title_name[0]}_heatmap_plot.png'), dpi=300, bbox_inches='tight')

        plt.savefig(os.path.join(save_figures_folder, f'{title_name[0]}_integrated_plot.png'), dpi=300, bbox_inches='tight')
        #plt.close(fig)
        if plot:
            plt.show()

            # Close to free memory
            plt.close(fig)
        #plt.show()

        # write python code below to 3d plot of azimuthal data: take q as x, y as filenumber, z as I
        # Example data structure for azimuthal_data
        # Replace this with your actual azimuthal data
        # azimuthal_data = [(q_array, I_array, err_array, tif_file), ...]

        # Extract data for plotting
        q_values = []
        intensities = []
        file_numbers = []

        for file_index, (q, I, *_rest) in enumerate(azimuthal_data):
            q_values.append(q)
            intensities.append(I)
            file_numbers.append(file_index)

        # Convert lists to numpy arrays
        q_grid = np.array(q_values)
        intensity_grid = np.array(intensities)
        file_numbers = np.array(file_numbers)

        # Generate a meshgrid for q and file numbers
        X, Y = np.meshgrid(q_grid[0], file_numbers)  # Assuming all q arrays are the same
        Z = intensity_grid

        # Create the 3D plot
        fig = plt.figure(figsize=(12, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Plot the surface
        surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

        # Add colorbar
        #colorbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
        #colorbar.set_label('Intensity')

        # Set axis labels
        ax.set_xlabel('q [nm⁻¹]')
        ax.set_ylabel('File Number')
        ax.set_zlabel('Intensity [a.u.]')
        #ax.set_xlim(5, 30)
        #set_plot_style(ax, 20, 'q [nm⁻¹]', 'Intensity [a.u.]')

        # Show the plot
        plt.tight_layout()
        if plot:
            plt.show()
        
        plt.close()



        # Assuming the azimuthal_data contains (q, I, *_rest)
        # azimuthal_data = [(q_array, I_array, err_array, tif_file), ...]

        # Extract data for plotting
        q_values = []
        intensities = []
        file_numbers = []

        for file_index, (q, I, *_rest) in enumerate(azimuthal_data):
            q_values.append(q)
            intensities.append(I)
            file_numbers.append(file_index)

        thickness = np.array(file_numbers)*10

        # Convert lists to numpy arrays
        q_grid = np.array(q_values)
        intensity_grid = np.array(intensities)
        thickness = np.array(thickness)

        # Generate a meshgrid for q (X-axis) and file numbers (Y-axis)
        X, Y = np.meshgrid(q_grid[0], thickness)  # Assuming all q arrays are the same length
        Z = intensity_grid  # This will hold the intensities for the heatmap

        # Create the figure
        fig, ax = plt.subplots(figsize=(8, 4))

        # Create the pcolormesh plot
        c = ax.pcolormesh(X, Y, Z, cmap='coolwarm', shading='auto')

        # Add colorbar
        # Create the colorbar without the fontsize argument
        colorbar = fig.colorbar(c, ax=ax, label='Intensity [a.u.]', orientation='vertical', pad=0.02)

        # Adjust the font size of the colorbar label
        colorbar.set_label('Intensity [a.u.]', fontsize=20)

        # Adjust the font size of the colorbar ticks
        colorbar.ax.tick_params(labelsize=20)  # Adjust tick label size



        # Set axis labels
        ax.set_xlabel('q [nm⁻¹]')
        ax.set_ylabel('Y [μm]')
        ax.set_xlim(5, 30)
        
        # remove 'thresh1_f_P3HB_02_50RPM_as_spun_Scan00001_s00020_00000.tif'
        # remore thresh1_ and _Scan00001_s00020_00000.tif from the filename

        title_name = [f.replace('thresh1_', '').replace('_s00000_00000.tif', '') for f in tif_files]

   
        ax.set_title(title_name[0], fontsize=16, y=1.05)
        set_plot_style(ax, 20, 'q [nm⁻¹]', 'Y [μm]')

        #set_plot_style(ax, 20, 'q [nm⁻¹]', '#index')

        # save the plot
        plt.savefig(os.path.join(save_figures_folder, f'{title_name[0]}_heatmap_plot.png'), dpi=300, bbox_inches='tight')

        plt.tight_layout()
        if plot:
            plt.show()
        plt.close()




## read HDF5 file


In [ ]:
def read_h5_data(base_path, samp_folder, keyword, azimuthal=True):
    """
    Read azimuthal or radial data from HDF5 files generated by integrate_tif_files.

    Parameters:
    -----------
    base_path : str
        Root path to the data.
    samp_folder : str
        Folder name corresponding to the sample.
    keyword : str
        Keyword like "PS_500C" etc.
    azimuthal : bool, optional
        If True, reads azimuthal data; otherwise reads radial data.

    Returns:    
    --------
    data_list : list of dict
        Each entry corresponds to one dataset stored in the HDF5 file.
    """
    import h5py
    import os
    import glob

    # Construct the expected folder path
    target_folder = os.path.join(base_path, 'OneD_integrated_WAXS_01', samp_folder)
    if not os.path.exists(target_folder):
        raise FileNotFoundError(f"Target folder does not exist: {target_folder}")

    # Locate HDF5 files
    pattern = f"*{keyword}*_azimuthal_data.h5" if azimuthal else f"*{keyword}*_radial_data.h5"
    h5_files = glob.glob(os.path.join(target_folder, pattern))

    if not h5_files:
        raise FileNotFoundError(f"No matching HDF5 files found in {target_folder} with pattern {pattern}")

    data_list = []

    for h5_path in h5_files:
        with h5py.File(h5_path, 'r') as hf:
            for key in hf:
                group = hf[key]
                data = {
                    "filename": group["filename"][()].decode() if isinstance(group["filename"][()], bytes) else group["filename"][()],
                    "real_date_time": group["real_date_time"][()].decode() if isinstance(group["real_date_time"][()], bytes) else group["real_date_time"][()],
                    "i0": group["i0"][()],
                    "i1": group["i1"][()],
                    "Photod": group["Photod"][()]
                }
                if azimuthal:
                    data.update({
                        "q": group["q"][()],
                        "I": group["I"][()],
                        "error": group["error"][()]
                    })
                else:
                    data.update({
                        "chi": group["chi"][()],
                        "I": group["I"][()]
                    })
                data_list.append(data)

    return data_list


# ex-situ


## base path and poni files

In [ ]:
import numpy as np
import os

# Base paths and file definitions
#base_path = '/Users/akmaurya/Desktop/20240628/in_situ'  

base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/'
poni_file = os.path.join(base_path, 'poni/poni_exsitu/LaB6_poni_New_01.poni')
#mask_file = os.path.join(base_path, 'poni/poni_exsitu/exsitu_mask_new_09.edf')
mask_file = os.path.join(base_path, 'poni/poni_insitu/mask_insitu_new_01.edf')
output_base_path = base_path  # Base path for the output directory structure

ai = pyFAI.load(poni_file) 
print (f"ai: {ai}")

## PHA3B

In [ ]:

from matplotlib.pyplot import plot
import numpy as np
# q-value range
q1 = 9.25  # Minimum q-value (1/nm)
q2 = 13 # Maximum q-value (1/nm)
vmin, vmax = 0, 15  # P3HB



folder = 'P3HB'

folder_path = os.path.join(base_path, folder)

# List of sample folders
keywords = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
# filter the keywords with given pattern
keywords = [f for f in keywords if 'Blank' in f]
print(f"folder_names: {keywords}")
#keywords=['cryomill_P3HB_powder_01_Scan00011','Blank_kaptontape_01_Scan00011','f_P3HB_02_50RPM_rep_as_spun_01_Scan00001','f_P3HB_03_100RPM_rep_as_spun_01_Scan00001','f_P3HB_04_200RPM_as_spun_01_Scan00005','f_P3HB_05_400RPM_as_spun_01_Scan00001','f_P3HB_06_600RPM_as_spun_01_Scan00001']



for keyword in keywords:

    samp_folder = f"{keyword}"
    print(f"Processing {samp_folder}...")

    integrate_tif_files(base_path, poni_file, mask_file,folder,keyword, output_base_path, nsave2d=100, azm_range = (-180, 0), q1=q1, q2=q2, vmin=vmin, vmax=vmax,plot=True)



## PET

In [ ]:

import numpy as np
# q-value range   #### value of (100) peak at 18.15
q1 = 17.9  # Minimum q-value (1/nm)
q2 = 18.9  # Maximum q-value (1/nm)
vmin, vmax = 0, 10  # P3HB
#(17.9, 18.5)
vmin, vmax = 0, 8  # PET



folder = 'P3HB'

folder_path = os.path.join(base_path, folder)

keywords = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
# filter the keywords with given pattern
keywords = [f for f in keywords if 'Blank' in f]
print(f"folder_names: {keywords}")

#keywords= ['cryomill_PET_powder_01_Scan00011','Run17_rep_f_PET_02_50rpm_as_spun_01_Scan00001','Run18_f_PET_03_100rpm_as_spun_01_Scan00001','Run19_f_PET_04_200rpm_as_spun_01_Scan00001','Run20_f_PET_05_400rpm_as_spun_01_Scan00001','Run21_f_PET_06_600rpm_as_spun_01_Scan00001']

#keywords = []

for keyword in keywords:

    samp_folder = f"{keyword}"
    print(f"Processing {samp_folder}...")

    integrate_tif_files(base_path, poni_file, mask_file,folder,keyword, output_base_path, nsave2d=1000, azm_range = (-180, -0), q1=q1, q2=q2, vmin=vmin, vmax=vmax,plot=True)



## PLLA

In [ ]:
from anyio import key
import numpy as np
# q-value range
q1 = 11  # Minimum q-value (1/nm)
q2 = 12.5  # Maximum q-value (1/nm)


vmin, vmax = 0,12  # Plla

folder = 'P3HB'

folder_path = os.path.join(base_path, folder)

# List of sample folders
keywords = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
#print(f"folder_names: {keywords}")
keywords = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
# filter the keywords with given pattern
keywords = [f for f in keywords if 'Blank' in f]
print(f"folder_names: {keywords}")

#keywords= ['cryomill_PLLA_powder_01_Scan00011','Run23_f_PLLA_02_50rpm_as_spun_01_Scan00001','Run24_f_PLLA_03_100rpm_as_spun_01_Scan00001','Run25_f_PLLA_04_200rpm_as_spun_01_Scan00001','Run26_f_PLLA_05_400rpm_as_spun_01_Scan00001','Run27_f_PLLA_06_600rpm_as_spun_01_Scan00001']
#keywords = []
#keywords=['cryomill_PLLA_powder_01_Scan00011']

for keyword in keywords:

    samp_folder = f"{keyword}"
    print(f"Processing {samp_folder}...")

    integrate_tif_files(base_path, poni_file, mask_file,folder,keyword, output_base_path, nsave2d=100, azm_range = (-105, -75), q1=q1, q2=q2, vmin=vmin, vmax=vmax, plot=True)


# Plot azimuthal integration

In [ ]:
import os
import matplotlib.pyplot as plt
import fnmatch

def plot_filtered_files(folder_path, patterns, xlim=(4, 20)):
    """
    Plot data from text files in a given folder, filtered by wildcard patterns in filenames.

    Parameters:
    - folder_path (str): Path to the folder containing the .txt files
    - patterns (list): List of wildcard patterns to filter filenames
    - xlim (tuple): x-axis limits for the plot
    """

    plt.figure(figsize=(6.5, 6), dpi=300)
    plt.clf()
    plt.ion()  # Enable interactive mode

    
    
    for filename in os.listdir(folder_path):
        
        if (
            filename.endswith('.txt') and
            'azimuthal' in filename and
            not filename.startswith('._') and
            any(fnmatch.fnmatch(filename, pattern) for pattern in patterns)
        ):
            file_path = os.path.join(folder_path, filename)


            try:
                q_values = []
                intensity_values = []

                with open(file_path, 'r') as f:
                    for line in f:
                        # Skip unwanted lines
                        if (
                            line.startswith('#') or
                            line.strip() == '' or
                            'dtype' in line or
                            'q:' in line or
                            'intensity:' in line
                        ):
                            continue

                        parts = line.strip().split()
                        if len(parts) >= 2:
                            q_values.append(float(parts[0]))
                            intensity_values.append(float(parts[1]))

                # Use part of the filename as label
                label = filename.split('.')[0]  # Adjust if needed
                print(f"Filename: {label}")
                label = label.replace('thresh1_', '')
                label = label.split('_Scan')[0]

                if 'f_' in label:
                    label = label.split('f_')[1]

                if 'P3HB' in label:
                    label = label.replace('P3HB', 'PHA-3B')

                if 'rep_' in label:
                    label = label.replace('rep_', '')
                label = label.replace('_01', '')
                #label = label.split('_f_')[1]
                
                # normlise intensity sum of total intensity
                intensity_values = np.array(intensity_values)
                intensity_values = 10000*intensity_values / np.sum(intensity_values)
                #intensity_values = intensity_values / np.max(intensity_values)

                
                plt.plot(q_values, intensity_values, '-o',label=label, markersize=2)

            except Exception as e:
                print(f"Failed to read {file_path}: {e}")
        #print(f"i: {i}")
    plt.xlabel('q [nm⁻¹]')
    plt.ylabel('Intensity [a.u.]')


    set_plot_style(plt.gca(), 20, 'q [nm⁻¹]', 'Intensity [a.u.]')
   


    plt.xlim(xlim)
    plt.legend()
    plt.tight_layout()
    plt.show()


folder = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/D_txt/'
#patterns = ['*PLLA_*s00010*', '*PLLA*_s000014*']

patterns = ['*cryomill*PLLA*_s00005*',
    '*PLLA*50rpm*s00010*',
    '*PLLA*100rpm*s00010*',
    '*PLLA*200rpm*s00010*',
    '*PLLA*400rpm*s00010*',
    '*PLLA*600rpm*s00010*',
    
]


plot_filtered_files(folder, patterns, xlim=(10, 15))

patterns = ['*cryomill_P3HB_powder*_s00005*',
 '*f_P3HB_*_50RPM*_s00010*',
 '*f_P3HB_*_100RPM*_s00012*',
 '*f_P3HB_*_200RPM*_Scan00005_s00010*',
 '*f_P3HB_*_400RPM*_s00005*',
 '*f_P3HB_*_600RPM*_s00007*'
]

plot_filtered_files(folder, patterns, xlim=(3, 22)) 

patterns = ['*cryomill_PET_powder*_s00010*',
 '*PET_*_50rpm*_s00013*',
 '*PET_*_100rpm*_s00004*',
 '*PET_*_200rpm*_s00001*',
 '*PET_*_400rpm*_s00007*',
 '*PET_*_600rpm*_s00005*'
]

plot_filtered_files(folder, patterns, xlim=(3, 35)) 



## Plot radial profile 

In [ ]:
import os
import fnmatch
import matplotlib.pyplot as plt

def read_and_plot_chi_intensity(folder_path, patterns, normalize=False):
    """
    Searches for files matching patterns in the folder, reads Chi vs I_radial data, and plots.
    
    Parameters:
    - folder_path (str): Path to the folder containing data files
    - patterns (list): List of wildcard patterns to match filenames
    - normalize (bool): If True, normalize the intensity values
    """
    plt.figure(figsize=(6.5, 6), dpi=300)
    plt.clf()
    plt.ion()  # Enable interactive mode

    for filename in os.listdir(folder_path):
        if (
            filename.endswith('.txt')
            and 'radial' in filename
            and not filename.startswith('._')
            and any(fnmatch.fnmatch(filename, pattern) for pattern in patterns)
        ):
            file_path = os.path.join(folder_path, filename)

            chi = []
            intensity = []

            try:
                with open(file_path, 'r') as file:
                    for line in file:
                        # Skip header or empty lines
                        if line.strip() == '' or line.startswith('#'):
                            continue

                        parts = line.strip().split()
                        if len(parts) >= 2:
                            chi.append(float(parts[0]))
                            intensity.append(float(parts[1]))

                if not chi:
                    print(f"No data found in {filename}. Skipping.")
                    continue
                # mask intesity in given chi mask in -82 to -75 but pass out of of this range
                chi = np.array(chi)
                intensity = np.array(intensity)
                mask = (chi <= -80) | (chi >= -74)
                chi = chi[mask]
                intensity = intensity[mask]
                # Optional normalization
                if normalize:
                    max_intensity = max(intensity)
                    #intensity = [i / max_intensity for i in intensity]

                # Normalize intensity by sum of total intensity
                intensity = np.array(intensity)
                intensity = 10000 * intensity / np.sum(intensity)

                # Use part of the filename as label
                #label = filename.split('_')[2]  # Adjust index if needed
                label = filename.split('.')[0]  # Adjust if needed
                print(f"Filename: {label}")
                label = label.replace('thresh1_', '')
                label = label.split('_01_Scan')[0]
                if 'f_' in label:
                    label = label.split('f_')[1]
                
                if 'P3HB' in label:
                    label = label.replace('P3HB', 'PHA-3B')

                if 'rep_' in label:
                    label = label.replace('rep_', '')
                label = label.replace('_01', '')
                
                plt.plot(chi, intensity,'-o',label=label, markersize=2)

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

    plt.xlabel('Chi (degrees)')
    plt.ylabel('Intensity (a.u.)')
    plt.xlim(-120, -60)
    #plt.ylim(2, 10)
    set_plot_style(plt.gca(), 20, 'Chi [degrees]', 'Intensity [a.u.]')
    plt.legend()
    plt.tight_layout()
    plt.show()



folder = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/D_txt/'
patterns = [#'*cryomill*PLLA*_s00005*',
    '*PLLA*50rpm*s00010*',
    '*PLLA*100rpm*s00010*',
    '*PLLA*200rpm*s00009*',
    '*PLLA*400rpm*s00014*',
    '*PLLA*600rpm*s00010*',
    
]
read_and_plot_chi_intensity(folder, patterns, normalize=True)    

patterns = [#'*cryomill_P3HB_powder*_s00010*',
 '*f_P3HB_*_50RPM*_s00010*',
 '*f_P3HB_*_100RPM*_s00012*',
 '*f_P3HB_*_200RPM*_Scan00005_s00010*',
 '*f_P3HB_*_400RPM*_s00005*',
 '*f_P3HB_*_600RPM*_s00007*'
]

read_and_plot_chi_intensity(folder, patterns, normalize=True) 

patterns = [#'*cryomill_PET_powder*_s00010*',
 '*PET_*_50rpm*_s00013*',
 '*PET_*_100rpm*_s00004*',
 '*PET_*_200rpm*_s00001*',
 '*PET_*_400rpm*_s00007*',
 '*PET_*_600rpm*_s00005*'
]

read_and_plot_chi_intensity(folder, patterns, normalize=True)     



# plot subtracted data

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import glob

def plot_subtracted_data(base_path, samp_folder, keyword, mode="azimuthal", pattern_name="subtracted"):
    """
    Reads and plots data from HDF5 files whose names match the keyword pattern.
    Also plots the dataset with the highest peak intensity from each file in a combined figure.

    Parameters:
    - base_path (str): Base directory path
    - samp_folder (str): Sample folder inside D1
    - keyword (str): Wildcard pattern to search for in filenames (e.g., '*P3HB*')
    - mode (str): 'azimuthal' or 'chi'
    - pattern_name (str): Prefix of the HDF5 file name (default: 'subtracted')
    """
    subtracted_dir = os.path.join(base_path, 'D1', samp_folder, 'Subtracted_Data')
    search_pattern = os.path.join(subtracted_dir, f"{pattern_name}_{keyword}_{mode}.h5")
    matching_files = glob.glob(search_pattern)

    if not matching_files:
        raise FileNotFoundError(f"No file matching pattern: {search_pattern}")

    summary_data = []  # Store (x, I, filename) of max peak intensity for each file

    for subtracted_file in matching_files:
        print(f"Plotting: {subtracted_file}")
        max_peak = -np.inf
        best_x = best_I = best_filename = None

        with h5py.File(subtracted_file, 'r') as hf:
            plt.figure(figsize=(6, 4))
            for key in hf.keys():
                group = hf[key]
                x = np.array(group["x"])
                I = np.array(group["I"])
                filename = group["filename"][()].decode("utf-8") if isinstance(group["filename"][()], bytes) else group["filename"][()]

                plt.plot(x, I, 'o-', markersize=1, label='Subtracted')
                plt.xlabel("q [1/nm]" if mode == "azimuthal" else "Chi [degrees]")
                plt.ylabel("Intensity [a.u.]")
                plt.title(f"{filename}")

                # Check if this dataset has the highest peak
                peak_intensity = np.max(I)
                if peak_intensity > max_peak:
                    max_peak = peak_intensity
                    best_x = x
                    best_I = I
                    best_filename = filename

            plt.tight_layout()
            plt.show()

        if best_x is not None and best_I is not None:
            summary_data.append((best_x, best_I, best_filename))

    # Plot summary of highest peak datasets
    if summary_data:
        plt.figure(figsize=(7, 5))
        for x, I, filename in summary_data:
            # Normalize intensity
            I = 10000 * I / np.sum(I)
            plt.plot(x+180, I, '-', label=os.path.basename(filename))
        plt.xlabel("q [1/nm]" if mode == "azimuthal" else "Chi [degrees]")
        plt.ylabel("Intensity [a.u.]")
        plt.title("Highest Intensity Peak from Each File")
        #plt.xlim(5, 25)
        set_plot_style(plt.gca(), 20, "q [1/nm]", "Intensity [a.u.]")
        #plt.legend(fontsize='small')
        plt.tight_layout()
        plt.show()


base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/'
samp_folder = 'P3HB'
print(f"Processing {samp_folder}...")

plot_subtracted_data(
    base_path=base_path,
    samp_folder=samp_folder,
    keyword="*P3HB*",
    mode="azimuthal"
)


plot_subtracted_data(
    base_path=base_path,
    samp_folder=samp_folder,
    keyword="*PET*",
    mode="azimuthal"
)

plot_subtracted_data(
    base_path=base_path,
    samp_folder=samp_folder,
    keyword="*PLLA*",
    mode="azimuthal"
)


# in-situ

In [ ]:
import os

# Base paths and file definitions
#base_path = '/Users/akmaurya/Desktop/20240628/in_situ'

base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/'
poni_file = os.path.join(base_path, 'poni/poni_insitu/LaB6_insitu_01.poni')
mask_file = os.path.join(base_path, 'poni/poni_insitu/mask_insitu_new_01.edf')
output_base_path = base_path  # Base path for the output directory structure


In [ ]:
# q-value range
q1 = 14  # Minimum q-value (1/nm)
q2 = 16  # Maximum q-value (1/nm)
vmin, vmax = 0, 2  # Define vmin and vmax as needed
folder = 'insitu'
# List of sample folders
keywords = [#'time_stamp_check_Scan00002', 
            #'insitu_Run1_LDPE_5mmmin_01_Scan00001', 
            'insitu_Run2_LDPE_5mmmin_01_Scan00001', 
            #'radiation_damage_abs101_elongatedLDPE_01_Scan00002', 
            #'radiation_damage_abs101_elongatedLDPE_01_Scan00003', 
            #'insitu_Run3_PHPD_film_base_5mmmin_01_Scan00001', 
            #'insitu_Run3_PHPD_film_base_5mmmin_01_Scan00002', 
            #'insitu_Run4_PHPD_film_base_5mmmin_01_Scan00001', 
            #'insitu_Run5_PHPD_film_base_5mmmin_01_Scan00001', 
            'insitu_Run6_PHPD_film_base_5mmmin_01_Scan00001', 
            #'insitu_Run7_PHPD_film_base_3mmmin_01_Scan00001', 
            'insitu_Run8_PHPD_film_base_3mmmin_01_Scan00001', 
            'insitu_Run9_PHPD_MBfilm_3mmmin_01_Scan00001', 
            'insitu_Run10_P5HV_film_3mmmin_01_Scan00001', 
            'insitu_Run11_air_with_TS600_01_Scan00001', 
            'insitu_Run11_nobeam_with_TS600_01_Scan00001', 
            #'insitu_Run12_LaB6_01_Scan00001', 
            #'insitu_Run12_LaB6_02_Scan00001', 
            'insitu_Run12_air_with_Scan00001', 
            'insitu_Run14_P5HV_film_3mmmin_01_Scan00001', 
            'insitu_Run15_P5HV_film_3mmmin_01_Scan00001', 
            'insitu_Run16_LDPE_film_3mmmin_01_Scan00001', 
            'insitu_Run17_P5HV_film_5mmmin_01_Scan00001']

for keyword in keywords:
    samp_folder = f"{keyword}"
    print(f"Processing {samp_folder}...")
    integrate_tif_files(base_path, poni_file, mask_file,folder,keyword, output_base_path, nsave2d=30, azm_range = (-45, 45), q1=q1, q2=q2, vmin=vmin, vmax=vmax)

